## Predicting Survival in the Titanic Data Set
<b>We will be using a decision tree to make predictions about the Titanic data set from Kaggle. This data set provides information on the Titanic passengers and can be used to predict whether a passenger survived or not.</b>

<b>Loading Data and modules</b>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn
from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn import tree, metrics, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [2]:
url= "https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv"
titanic = pd.read_csv(url)

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
titanic.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']

In [5]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

<b>You use only Pclass, Sex, Age, SibSp (Siblings aboard), Parch (Parents/children aboard) and Fare to predict whether a passenger survived.</b>

In [6]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
#Creating another DataFrame for data manipulation
titanic_data = titanic

In [10]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
#Checking for Null Values
titanic_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [16]:
#Imputing null values in the Age field with mean Age.
titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace = True)

In [17]:
titanic_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [18]:
#Binning the Age into different bands to convert the continous variable to categorical variable.
titanic_data['Age_band']=0
titanic_data.loc[titanic_data['Age']<=16,'Age_band']=0
titanic_data.loc[(titanic_data['Age']>16)&(titanic_data['Age']<=32),'Age_band']=1
titanic_data.loc[(titanic_data['Age']>32)&(titanic_data['Age']<=48),'Age_band']=2
titanic_data.loc[(titanic_data['Age']>48)&(titanic_data['Age']<=64),'Age_band']=3
titanic_data.loc[titanic_data['Age']>64,'Age_band']=4

In [19]:
#Adding the SibSp and ParCh fields to see if the passenger travelled alone or with a family.
titanic_data['Family_Size']=0
titanic_data['Family_Size']=titanic_data['Parch']+titanic_data['SibSp']#family size
titanic_data['Alone']=0
titanic_data.loc[titanic_data.Family_Size==0,'Alone']=1#Alone

In [21]:
#Binning the Fare into different categories to convert the continous variable to categorical variable.
titanic_data['Fare_cat']=0
titanic_data.loc[titanic_data['Fare']<=7.91,'Fare_cat']=0
titanic_data.loc[(titanic_data['Fare']>7.91)&(titanic_data['Fare']<=14.454),'Fare_cat']=1
titanic_data.loc[(titanic_data['Fare']>14.454)&(titanic_data['Fare']<=31),'Fare_cat']=2
titanic_data.loc[(titanic_data['Fare']>31)&(titanic_data['Fare']<=513),'Fare_cat']=3

In [22]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_band,Family_Size,Alone,Fare_cat
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,1,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,1,0,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,1,0,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,2,0,1,1


One Hot Encoding for Pclass, Sex, Age and Fare categories

In [23]:
cat_cols =  ["Pclass", "Sex", "Age_band","Fare_cat"]

In [25]:
for col in cat_cols:
    dummies = pd.get_dummies(titanic_data[col], prefix = col)
    titanic_data = titanic_data.join(dummies)
    titanic_data = titanic_data.drop(col, axis = 1)

In [26]:
titanic_data.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,Sex_male,Age_band_0,Age_band_1,Age_band_2,Age_band_3,Age_band_4,Fare_cat_0,Fare_cat_1,Fare_cat_2,Fare_cat_3
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,...,1,0,1,0,0,0,1,0,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,...,0,0,0,1,0,0,0,0,0,1
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,...,0,0,1,0,0,0,0,1,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,...,0,0,0,1,0,0,0,0,0,1
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,...,1,0,0,1,0,0,0,1,0,0


In [29]:
titanic_data.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'Family_Size', 'Alone', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Age_band_0',
       'Age_band_1', 'Age_band_2', 'Age_band_3', 'Age_band_4', 'Fare_cat_0',
       'Fare_cat_1', 'Fare_cat_2', 'Fare_cat_3'],
      dtype='object')

Dropping the fields that will not be useful for prediction

In [30]:
titanic_data.drop(['PassengerId','Name','Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'], axis = 1, inplace = True)

In [31]:
titanic_data.columns

Index(['Survived', 'Family_Size', 'Alone', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Sex_female', 'Sex_male', 'Age_band_0', 'Age_band_1', 'Age_band_2',
       'Age_band_3', 'Age_band_4', 'Fare_cat_0', 'Fare_cat_1', 'Fare_cat_2',
       'Fare_cat_3'],
      dtype='object')

In [32]:
y = titanic_data.pop('Survived')

In [34]:
X = titanic_data

In [35]:
X.shape

(891, 16)

In [36]:
y.shape

(891,)

Creating Test/Train Split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Running the Decision Tree model on the Train set

In [38]:
dtree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
dtree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [39]:
y_pred = dtree.predict(X_test)

In [40]:
# how did our model perform?
count_misclassified = (y_test != y_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 51
Accuracy: 0.81


We are getting an accuracy of 81%. Lets try to tune the hyper parameters.

In [41]:
dtree2 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)
dtree2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [42]:
y_pred2 = dtree2.predict(X_test)

In [43]:
# how did our model perform?
count_misclassified = (y_test != y_pred2).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_test, y_pred2)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 53
Accuracy: 0.80


Accuracy has dropped to 80%. Lets try decreasing the max_depth to 2

In [44]:
dtree3 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=0)
dtree3.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [45]:
y_pred3 = dtree3.predict(X_test)

In [46]:
# how did our model perform?
count_misclassified = (y_test != y_pred3).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_test, y_pred3)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 57
Accuracy: 0.79


Accuracy has dropped further to 79%. The initial model gives the best accuracy. Lets try applying Cross validation using the initial model.

In [48]:
from sklearn.model_selection import cross_val_score

In [51]:
scores = cross_val_score(dtree, X, y, scoring='accuracy', cv=10)
scores, scores.mean()

(array([0.81111111, 0.82222222, 0.78651685, 0.84269663, 0.83146067,
        0.78651685, 0.80898876, 0.78651685, 0.83146067, 0.80681818]),
 0.8114308818522302)

We are getting a similar accuracy using K-Fold cross validation suggesting that the model is robust.